In [7]:
import PyPDF2
import pdfplumber as pr
import pandas as pd
from openpyxl import load_workbook
import re
import os

In [6]:
PDF_files  = []
input_path = r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/input'
for x in os.listdir(input_path):
    if '.pdf' in x:
        PDF_files.append(x)

GFI  = pd.read_excel(r'/Users/andyhsu/Documents/GitHub/Intern_Portfolio/Intern_Portfolio/政府基金運用情形/input/202111 政府基金運用資訊 v2.0.xlsx',sheet_name = 'Government Fund Info (Raw Data)',skiprows = 2)
# 
#把index處理一下
new_index = []
old_index = GFI.set_index('Unnamed: 0').index
for i in GFI.set_index('Unnamed: 0').index:
    #只抓前面的中文部分
    if re.match('[\u4e00-\u9fa5]+',i):
        new_index.append(re.match('[\u4e00-\u9fa5]+',i)[0])

In [ ]:
#因為最後excel跟PDF的欄名可能不一樣，我們這邊用DICT的方式做一個轉換
def convert_勞退_dict(x):
    勞退_dict = {
    '公債、公司債、金融債券及特別股':'公債',
    '股票及受益憑證投資（含期貨）':'股票及受益憑證投資',
    '國內委託經營':'國內委託',
    '國外委託經營':'國外委託',
    '合           計':'金額總計'
    }
    if x in 勞退_dict:
        result = 勞退_dict[x]
    else:
        result = x
    return result

def convert_勞保_dict(x):
    勞保_dict = {
    '公債、公司債、金融債券及特別股':'公債',
    '股票及受益憑證投資（含期貨）':'股票及受益憑證投資',
    '國內委託經營':'國內委託',
    '國外委託經營':'國外委託',
    '合    計':'金額總計'
    }
    if x in 勞保_dict:
        result = 勞保_dict[x]
    else:
        result = x
    return result

def convert_國保_dict(x):
    國保_dict = {
    '合           計':'金額總計',
    '公債、公司債、金融債券及特別股':'公債',
    '股票及受益憑證投資（含期貨）':'股票及受益憑證投資'
    }
    if x in 國保_dict:
        result = 國保_dict[x]
    else:
        result = x
    return result

def convert_退撫_dict(x):
    退撫_dict = {
    '自行運用小計':'自行運用',
    '國內委託經營':'國內委託',
    '國外委託經營':'國外委託',
    '委託經營小計':'委託經營',
    '合                 計':'金額總計'
}
    if x in 退撫_dict:
        result = 退撫_dict[x]
    else:
        result = x
    return result

In [10]:
for pdf_file in PDF_files:
    if '國民年金' in pdf_file:
        國保_pdf = pr.open(input_path+'/'+pdf_file)
    elif '勞工退休' in pdf_file:
        勞退_pdf = pr.open(input_path+'/'+pdf_file)
    elif '基金運用' in pdf_file:
        退撫_pdf = pr.open(input_path+'/'+pdf_file)
    elif '勞工保險' in pdf_file:
        勞保_pdf = pr.open(input_path+'/'+pdf_file)

In [ ]:
#輸入幫我們讀取所有的圖表
def get_all_table(file):
    tables_dict = {}
    ps = file.pages
    print(len(ps))
    i = 0
    for p in range(len(ps)):
        #進去每一頁
        pg = ps[p]
        #找到有多少表格
        tables = pg.extract_tables()
        #一頁只有一個表格的狀況
        if len(tables) == 1:
            table = tables
            tables_dict['表格'+str(i)] = table
            i += 1
        #一頁有超過一個表格的狀況
        elif len(tables) > 1:
            for g in range(len(tables)):
                table = tables[g]
                tables_dict['表格'+str(i)] = table
                i += 1
    return tables_dict

#依照我們設定的行數來找尋目標table
def get_target_table(tables_dict,target_len):
    i = 0
    #用來放目標table的
    target_table_df = pd.DataFrame({})
    #用表格的長度來決定要抓什麼表格
    for table in list(tables_dict.values()):
        if len(table) == target_len:
            print(i)
            #print(pd.DataFrame(table[1:],columns = table[0]))
            Table = pd.DataFrame(table[1:],columns = table[0])
            target_table_df = pd.concat([target_table_df,Table],axis=1)
            
        #這邊是發現有的表格會變成三維的，所以做特殊處理
        elif len(table[0]) == target_len:
            print(i)
            #print(pd.DataFrame(table[0][1:],columns = table[0][0]))
            Table = pd.DataFrame(table[0][1:],columns = table[0][0])
            #因為項目我已經有了，所以要拿掉
            target_table_df = pd.concat([target_table_df,Table],axis=1)
        i += 1
    return target_table_df

#勞退之後續處理
#放到主要程式碼時，前面要加一個判斷這個檔案的名稱(勞退勞保or年金等)，來決定後續的動作
def handle_勞退_process(DataFrame):
    DataFrame.columns = ['項         目','勞退新制\nLPF','占基金運用比率（％）新','項         目','勞退舊制\nLRF','占基金運用比率（％）舊']
    #去掉欄重複的
    新舊制勞退_df = DataFrame.loc[:,~DataFrame.columns.duplicated()]
    新舊制勞退_df['項         目'] = 新舊制勞退_df['項         目'].apply(lambda x:convert_勞退_dict(x))   
    new_DF = pd.DataFrame(['-']*len(old_index),index = old_index,columns = ['對照欄位(之後要移除)'])
    new_DF['對照欄位(之後要移除)'] = new_index
    new_DF.reset_index()
    勞退完成_df = pd.merge(left = new_DF.reset_index(),right = 新舊制勞退_df,left_on = '對照欄位(之後要移除)',right_on = '項         目',how='outer')
    #這邊看之後能不能改的比較自動
    勞退完成_df = 勞退完成_df.iloc[[0,1,2,3,4,5,6,7,8,9,10,11,15,19,23,24,25,12,16,20,26],[0,3,4,5,6]]
    勞退完成_df.drop_duplicates(inplace=True)
    勞退完成_df.reset_index(drop=True,inplace=True)
    #加上%符號
    勞退完成_df['占基金運用比率（％）新'] = 勞退完成_df['占基金運用比率（％）新'] + '%'
    勞退完成_df['占基金運用比率（％）舊'] = 勞退完成_df['占基金運用比率（％）舊'] + '%'
    return 勞退完成_df

def handle_勞保_process(DataFrame):
    DataFrame.columns = ['項         目','勞保\nLIF','%']
    #去掉欄重複的
    #這邊不會用到
    勞保_df = DataFrame.loc[:,~DataFrame.columns.duplicated()]
    勞保_df['項         目'] = 勞保_df['項         目'].apply(lambda x:convert_勞保_dict(x))   
    new_DF = pd.DataFrame(['-']*len(old_index),index = old_index,columns = ['對照欄位(之後要移除)'])
    new_DF['對照欄位(之後要移除)'] = new_index
    new_DF.reset_index()
    勞保完成_df = pd.merge(left = new_DF.reset_index(),right = 勞保_df,left_on = '對照欄位(之後要移除)',right_on = '項         目',how='outer')
    #這邊看之後能不能改的比較自動
    勞保完成_df.drop_duplicates(inplace=True)
    勞保完成_df_1 = 勞保完成_df.reset_index(drop=True)
    勞保完成_df_1 = 勞保完成_df_1.iloc[[0,1,2,3,4,5,6,7,8,9,10,11,13,15,17,18,19,12,14,16,20],[0,3,4]]
    #加上%符號
    勞保完成_df_1['%'] = 勞保完成_df_1['%'] + '%'
    return 勞保完成_df_1

def handle_國保_process(DataFrame):
    DataFrame.columns = ['項         目','國保基金\nNPIF','%']
    #去掉欄重複的
    #這邊不會用到
    國保_df = DataFrame.loc[:,~DataFrame.columns.duplicated()]
    國保_df['項         目'] = 國保_df['項         目'].apply(lambda x:convert_國保_dict(x))  
    new_DF = pd.DataFrame(['-']*len(old_index),index = old_index,columns = ['對照欄位(之後要移除)'])
    new_DF['對照欄位(之後要移除)'] = new_index
    new_DF.reset_index()
    國保完成_df = pd.merge(left = new_DF.reset_index(),right = 國保_df,left_on = '對照欄位(之後要移除)',right_on = '項         目',how='outer')
    #這邊看之後能不能改的比較自動
    國保完成_df.drop_duplicates(inplace=True)
    國保完成_df_1 = 國保完成_df.reset_index(drop=True)
    國保完成_df_1 = 國保完成_df_1.iloc[[0,1,2,3,4,5,6,7,8,9,10,11,13,15,17,18,19,12,14,16,20],[0,3,4]]
    #加上%符號
    國保完成_df_1['%'] = 國保完成_df_1['%'] + '%'
    return 國保完成_df_1

def handle_退撫_process(DataFrame):
    DataFrame.columns = ['項         目','退撫基金\nPSPF','%']
    #去掉欄重複的
    #這邊不會用到
    退撫_df = DataFrame.loc[:,~DataFrame.columns.duplicated()]
    退撫_df['項         目'] = 退撫_df['項         目'].apply(lambda x:convert_退撫_dict(x))  
    new_DF = pd.DataFrame(['-']*len(old_index),index = old_index,columns = ['對照欄位(之後要移除)'])
    new_DF['對照欄位(之後要移除)'] = new_index
    new_DF.reset_index()
    退撫完成_df = pd.merge(left = new_DF.reset_index(),right = 退撫_df,left_on = '對照欄位(之後要移除)',right_on = '項         目',how='left')
    #這邊看之後能不能改的比較自動
    #這邊一定不要做drop_duplicates，因為都沒有資料，刪掉會出錯
    #退撫完成_df.drop_duplicates(inplace=True)
    退撫完成_df_1 = 退撫完成_df.reset_index(drop=True)
    退撫完成_df_1 = 退撫完成_df_1.iloc[:,[0,3,4]]
    #加上%符號
    退撫完成_df_1['%'] = 退撫完成_df_1['%'] + '%'
    return 退撫完成_df_1

In [ ]:
#勞退文件的所有table
勞退_all_table = get_all_table(勞退_pdf)
#要改
勞退_df  = get_target_table(勞退_all_table,17)
勞退_df_final = handle_勞退_process(勞退_df)
勞退_df_final

In [ ]:
勞保_all_table = get_all_table(勞保_pdf)
#要改
勞保_df  = get_target_table(勞保_all_table,20)
勞保_df_final = handle_勞保_process(勞保_df)
勞保_df_final

In [ ]:
國保_all_table = get_all_table(國保_pdf)
#要改
國保_df  = get_target_table(國保_all_table,18)
國保_df_final = handle_國保_process(國保_df)
國保_df_final

In [ ]:
退撫_all_table = get_all_table(退撫_pdf)
#要改
退撫_df  = get_target_table(退撫_all_table,14)
退撫_df_final = handle_退撫_process(退撫_df)
退撫_df_final

In [ ]:
final_df = pd.concat([勞退_df_final,勞保_df_final.iloc[:,1:],國保_df_final.iloc[:,1:],退撫_df_final.iloc[:,1:]],axis=1)
outpath = 'output'
final_df.to_csv(r'\Users\andyhsu\Documents\GitHub\Intern_Portfolio\Intern_Portfolio\政府基金運用情形\output\政府基金資料page1.csv',index=False,header=True,encoding='utf-8-sig')